In [1]:
import os
import torch
from tqdm import tqdm


from preprocessor import AgeRecognitionPreprocessor
from dataset import AgeRecognitionDataset
from models import resent101_age_recogniser

In [2]:
IMAGE_DIR = './Cleaned/'
TEST_PAIRINGS = './test_data.csv'
VARIANT = 'resnet101'
DEVICE = 'cuda'

In [3]:
model = resent101_age_recogniser().to(DEVICE)
preprocessor = AgeRecognitionPreprocessor()
dataset = AgeRecognitionDataset(triplet_csv_path=TEST_PAIRINGS, image_dir=IMAGE_DIR, preprocessor=preprocessor, kfolds=1, device=DEVICE)

In [4]:
if os.path.exists(f'./Checkpoint/{VARIANT}/best.pt'):
    print("Loading best ver...")
    best_state = torch.load(f'./Checkpoint/{VARIANT}/best.pt')
    model.load_state_dict(best_state['model_state_dict'])

Loading best ver...


In [5]:
from torch import cosine_similarity

result = []
model.eval()    
total_loss = 0
for item in tqdm(dataset):
    # Batch shape: (N, Anchor-Positive-Negative, C, H, W)
    predictions = model.forward_features(item)
    anchor_pos = (cosine_similarity(predictions[0].unsqueeze(0), predictions[1].unsqueeze(0)).cpu().detach().item() + 1) / 2
    anchor_neg = (cosine_similarity(predictions[0].unsqueeze(0), predictions[2].unsqueeze(0)).cpu().detach().item() + 1) / 2
    pos_neg = (cosine_similarity(predictions[1].unsqueeze(0), predictions[2].unsqueeze(0)).cpu().detach().item() + 1) / 2
    result.append([anchor_pos, anchor_neg, pos_neg])

100%|██████████| 49828/49828 [41:54<00:00, 19.82it/s]  


In [6]:
import pandas as pd

similarities = pd.DataFrame(result, columns=['anchor_pos', 'anchor_neg', 'pos_neg'])
similarities

,anchor_pos,anchor_neg,pos_neg
0,0.333707,0.377810,0.398726
1,0.861512,0.470617,0.412320
2,0.970144,0.404495,0.332855
3,0.961106,0.461787,0.376850
4,0.928912,0.288522,0.324836
...,...,...,...
49823,0.832842,0.539848,0.483050
49824,0.843990,0.589229,0.552761
49825,0.928485,0.431824,0.462893
49826,0.642524,0.917398,0.576163


In [8]:
similarities.to_csv('./test_results_resnet101.csv')

Accuracy computation under given threshold

In [1]:
import pandas as pd
import numpy as np

similarities = pd.read_csv('./test_results_resnet101.csv', index_col=0)
similarities

,anchor_pos,anchor_neg,pos_neg
0,0.333707,0.377810,0.398726
1,0.861512,0.470617,0.412320
2,0.970144,0.404495,0.332855
3,0.961106,0.461787,0.376850
4,0.928912,0.288522,0.324836
...,...,...,...
49823,0.832842,0.539848,0.483050
49824,0.843990,0.589229,0.552761
49825,0.928485,0.431824,0.462893
49826,0.642524,0.917398,0.576163


In [2]:
acc_report = []

In [3]:
THRESHOLD = 0.65

In [4]:
anchor_pos_np = similarities['anchor_pos'].to_numpy()
anchor_neg_np = similarities['anchor_neg'].to_numpy()
pos_neg_np = similarities['pos_neg'].to_numpy()

In [5]:
def compute_acc(np_arr, higher, lower):
    '''
    Assigns higher to values above threshold and lower to values below threshold.
    So for positive pairings: 1, 0; negative pairings: 0, 1
    '''
    correct = np.where(np_arr >= THRESHOLD, higher, lower).sum()
    wrong = len(np_arr) - correct
    acc = correct / len(np_arr)
    print(correct, wrong, acc)
    return correct, wrong, acc

In [6]:
acc_report.append(compute_acc(anchor_pos_np, 1, 0))

39990 9838 0.8025608091835915


In [7]:
acc_report.append(compute_acc(anchor_neg_np, 0, 1))

40751 9077 0.8178333467126917


In [8]:
acc_report.append(compute_acc(pos_neg_np, 0, 1))

40726 9102 0.8173316207754676


In [9]:
acc_report_df = pd.DataFrame(acc_report, columns=['correct', 'wrong', 'accuracy'])
acc_report_df

,correct,wrong,accuracy
0,39990,9838,0.802561
1,40751,9077,0.817833
2,40726,9102,0.817332


In [10]:
overall_acc = acc_report_df.correct.sum() / (acc_report_df.correct.sum() + acc_report_df.wrong.sum())
overall_acc

0.8125752588905836

In [11]:
tp = acc_report_df.iloc[0].correct
tn = acc_report_df.iloc[1].correct + acc_report_df.iloc[2].correct
fp = acc_report_df.iloc[1].wrong + acc_report_df.iloc[2].wrong
fn = acc_report_df.iloc[0].wrong
tn, fp, fn, tp

(81477.0, 18179.0, 9838.0, 39990.0)

Using sklearn classification report

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

arr = np.where(anchor_pos_np >= THRESHOLD, 1, 0)
arr = np.append(arr, np.where(pos_neg_np >= THRESHOLD, 1, 0))
y_predict = np.append(arr, np.where(anchor_neg_np >= THRESHOLD, 1, 0))

In [13]:
y_true = np.array([1 if i < len(anchor_pos_np) else 0 for i in range(len(anchor_pos_np) * 3)])

In [14]:
y_predict

array([0, 1, 1, ..., 0, 1, 1])

In [15]:
y_true

array([1, 1, 1, ..., 0, 0, 0])

In [16]:
print(classification_report(y_true, y_predict))

              precision    recall  f1-score   support

           0       0.89      0.82      0.85     99656
           1       0.69      0.80      0.74     49828

    accuracy                           0.81    149484
   macro avg       0.79      0.81      0.80    149484
weighted avg       0.82      0.81      0.82    149484



In [17]:
print(confusion_matrix(y_true, y_predict))

[[81477 18179]
 [ 9838 39990]]


Triplet Matching Evaluation

In [18]:
max_indices = similarities.to_numpy().argmax(axis=1)
correct = np.where(max_indices == 0, 1, 0).sum() # 0th index is the anchor-positive pair, which is the desired matching
acc = correct / len(max_indices)
acc

0.8062334430440716